# OpenStreetMap Case Study

## Map Area

    Manhattan, New York, United States

    http://overpass-api.de/api/map?bbox=-74.0133,40.7245,-73.9574,40.7734
    
    From the link above, I downloaded the data of midtown Manhattan. Since I have worked and lived in the New York City for several years, I choose this area to do this case study.

## Problems Encountered in the Map

After investigating the data in sqlite. I found some problems with the street names and the postcodes. I used pandas to read the csv file to clean the street names and postcodes.I will not touch the original xml here. Because, it takes time to turn the xml fime into csv. If the file is bigger, it will take more time. It's an timely expensive calculation. I will use the csv files to do the next cleaning step. So, I use pandas to read the file into a data frame. Then, I invested the data, corrected the data, and save the data back to a new csv file.

- Abbreviated street names('W 35th St')
- Postcodes with irregular formats('NY 10001', '10001-2062'
       

### Abbreviated street names

I used pandas to iter the rows to correct the street names.

In [ ]:
#Check the street name type
short_street_name = set()
for index, row in df_ways_tags.iterrows():
    if row['key'] == 'street':
        street_value = row['value']
        street_type = street_value.split(' ')[-1]
        short_street_name.add(street_type)
        
print(short_street_name)

In [ ]:
#Get weird street name
weird_street_name = ['D','Americas','633','Bowery','A', 'St', 'B', 'C']
street_name = set()
for index, row in df_ways_tags.iterrows():
    if row['key'] == 'street' and  row['value'].split(' ')[-1] in weird_street_name:
        street_name.add(row['value'])
print(street_name)

In [ ]:
#Correct the street name
for index, row in df_ways_tags.iterrows():
    if row['key'] == 'street' and row['value'] == 'W 35th St':
        print('Made change to the row of index %s' % index)
        df_ways_tags.set_value(index, 'value', 'W 35th Street')

The street name of 'W 35th St' is changed into 'W 35th Street'.

### Postcodes with irregular formats

In [ ]:
#looking for postcodes with irregular formats
import re
bad_postcode = []
postcode = re.compile(r'^\d{5}$')
for index, row in df_ways_tags.iterrows():
    if row['key'] == 'postcode' and row['value']:
        m = postcode.match(row['value'])
        if not m:
            bad_postcode.append((index, row['value']))
print(bad_postcode)

In [ ]:

import re
get_postcode = re.compile(r'(?<!\d)\d{5}(?!\d)')

#Correct the postcode
for index, postcode in bad_postcode:
    if postcode:
        m2 = get_postcode.search(postcode)
        if m2:
            print("%s -> %s" % (postcode, m2.group(0)))
            df_ways_tags.set_value(index, 'value', m2.group(0))
        else:
            print("can't fix %s, set as None" % postcode)
            df_ways_tags.set_value(index, 'value', None)

The code above correct the postcodes. The changes are listed below:
- NY 10001 -> 10001
- 10001-2062 -> 10001
- 10019-9998 -> 10019
- can't fix 83, set as None
- NY 10016 -> 10016
- New York, NY 10065 -> 10065

## Use the command line to get rid of the header of the csv files
```
tail -n +2 nodes.csv > nodes_no_head.csv
tail -n +2 nodes_tags.csv > nodes_tags_no_head.csv
tail -n +2 ways.csv > ways_no_head.csv
tail -n +2 ways_tags_clean.csv > ways_tags_no_head.csv
tail -n +2 ways_nodes.csv > ways_nodes_no_head.csv
```

## Overview statistics of the dataset 

### Size of the file

- NYMap.xml ................... 59.3 MB
- nodes.csv ................... 17.5 MB
- nodes_tags.csv .............. 2.7 MB
- ways.csv .................... 2.1 MB
- ways_tags.csv ............... 6.7 MB
- ways_nodes.csv .............. 7.5 MB

### Number of unique users
```
sqlite> SELECT
   ...> (SELECT count( DISTINCT uid) FROM nodes)
   ...> +
   ...> (SELECT count( DISTINCT uid) FROM ways);
```
1507

### Number of nodes
```
sqlite> SELECT COUNT(DISTINCT id) FROM nodes;
```
194960

### Number of ways
```
sqlite> SELECT COUNT(DISTINCT id) FROM ways;
```
34031

### Overview information abount ways
```
sqlite> SELECT key, count(*) FROM ways_tags
   ...> GROUP BY key
   ...> ORDER BY count(*) DESC
   ...> LIMIT 20;
```
- height,26742
- building,18652
- bin,18348
- street,14122
- housenumber,14105
- postcode,13927
- material,13788
- colour,12076
- shape,9333
- part,8661
- name,5033
- highway,4916
- levels,2814
- city,1941
- oneway,1928
- county,1898
- cfcc,1891
- name_base,1787
- min_height,1761
- name_type,1659

We can see there're buiding, height, street and other information about ways.


## Additional findings


### Top 5 most popular cuisine
```
sqlite> SELECT value, count(*) FROM nodes_tags
   ...> WHERE key = 'cuisine'
   ...> GROUP BY value
   ...> ORDER BY count(*) DESC LIMIT 5;
```
- coffee_shop,104
- italian,73
- pizza,67
- burger,55
- american,54

The top 5 most popular cuisine are coffee, italian, pizza, burger, american. That's quite true, according to my living experience in New York. There're coffee shops everywhere.

### Top 5  most popular shop
```
sqlite> SELECT value, count(*) FROM nodes_tags
   ...> WHERE key = 'shop'
   ...> GROUP BY value
   ...> ORDER BY count(*) DESC LIMIt 5;
```
- clothes,202
- convenience,73
- supermarket,58
- bakery,46
- shoes,46

The top 5 most popular shops are clothes shops, conveniences shops, supermarkets, bakeries and shoes. The number of shops are big. That's New York, a heaven for shopping.

### Places to shop
```
sqlite> WITH sub AS(
   ...>    SELECT ways_nodes.id, ways_nodes.node_id, nodes_tags.key AS nodes_key, nodes_tags.value AS nodes_value,
   ...>     ways_tags.key AS ways_key, ways_tags.value AS ways_value
   ...>    FROM ways_nodes
   ...>    LEFT JOIN nodes
   ...>    ON ways_nodes.node_id = nodes.id
   ...>    LEFT JOIN nodes_tags
   ...>    ON nodes.id = nodes_tags.id
   ...>    LEFT JOIN ways
   ...>    ON ways_nodes.id = ways.id
   ...>    LEFT JOIN ways_tags
   ...>    ON ways.id = ways_tags.id
   ...>    WHERE nodes_tags.key = 'shop')
   ...> SELECT sub.ways_value from sub
   ...> WHERE sub.ways_key ='street';
```
"Extra Place"
"Broadway"
"East 37th Street"

No surprisingly, the 'Broadway' and 'East 37th Street are good areas for shopping.

### Tourism 
```
sqlite> SELECT value, count(*) FROM nodes_tags
   ...> WHERE key = 'tourism'
   ...> GROUP BY value
   ...> ORDER BY count(*) DESC LIMIT 5;
```
- hotel,101
- attraction,56
- artwork,40
- museum,21
- information,8

From the data, we can find out that New York is good place to visit. There're so many hotels, attractions, museums and etc. 

### User who made the biggiest contribution

I am curious who is contributing for the OpenStreet Map website. Here, I would like who submit the most contribution.

```
sqlite> SELECT uid, count(*) FROM nodes
   ...> group by uid
   ...> order by count(*) desc
   ...> LIMIT 1;
```
uid :1781294, number of records: 67455

The user with the user id of 1781294 made the biggest contribution in editing nodes. He got 67455 records.

```
sqlite> WITH sub AS(
   ...> SELECT nodes_tags.id, nodes_tags.key, nodes_tags.value, nodes.uid FROM nodes_tags
   ...> LEFT JOIN nodes
   ...> ON nodes_tags.id = nodes.id)
   ...> SELECT uid, key, count(*)
   ...> FROM sub
   ...> WHERE uid = 1781294
   ...> GROUP BY key
   ...> ORDER BY count(*) DESC
   ...> LIMIT 5;
```
- 1781294,housenumber,5528
- 1781294,street,5526
- 1781294,postcode,5524
- 1781294,name,71
- 1781294,amenity,56

Here, we can see. He made so many contribution for the housenumber, street and the postcode. The OpenStreet Map website can't be successful without users like this.


# Conclusion

With this project, I investigated the data, cleaned the data, turns the data into different formats. So many tools are used in this project - Python, sqlite, pandas, re and command line. It was a good practice for me. The data shows exactly what New York is. It's a place good to visit, good for shopping and full of good restaurants. It's all because the contributors of the data, so that we can get this information. I will make my contribution later on.

### Suggestions for improving the data or its analysis
I think OpenSteet can work with Yelp to add more details about restaurants and other places

- Benefits:
    - OpenStreet Map can get a lot of info directly
    - People will use OpenStreet Map more oftern, since it has more information
    
- Anticipated Problems:
    - It will be very complexe when merge the data from two companies
    - It will be hard to get data from a big company like Yelp